### Designing a finite state automaton in Python

CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
February 16, 2023
***

The textbook defines a Finite State Automaton (FSA) as a tuple (see Def 3 in section 13.3.3). Think of a _tuple_ as an ordered pair with more than two elements. The tuple for an FSA is $(S,I,s_0,F,f)$ where 
* A finite set of states $S$
* A set of input characters $I$
* A start state $s_0$
* A set of final or accept states $F$
* A transition function $f:S\times I \rightarrow S$
<br>


We often draw FSAs by representing each state $s\in S$ with a circle, representing all states in $F$ with a double circle, representing the start state by a circle with an arrow pointing toward it, and representing transition functions as arrows connecting one state to another. The label of the arrow connecting one state to another is the input $i\in I$ that causes the FSA to _transition_from one state to another. 

Consider the following FSA

![colon-dash FSA](colon-dash-fsa.png)

The elements of the FSA tuple above are
* The finite set of states is $S=\{s_0,s_1,s_2,s_{\rm err}\}$
* The finite set of input characters is $I=\{{\rm any\  keyboard\  character}\}$
* The start state is $s_0$
* The set of final or accept states is $F=\{s_2\}$
* The transition function $f:S\times I \rightarrow S$ are all the states connected by the labeled arrows:

| present state | input             | next state |
| :-:           | :-:               | :-:        |
| $s_0$         | ':'               | $s_1$      |
| $s_1$         | '-'               | $s_2$      |
| $s_2$         | anything          | $s_2$      |
| $s_0$         | anything but ':'  | $s_{err}$  |
| $s_1$         | anything but '-'  | $s_{err}$  |
| $s_{err}$     | anything          | $s_{err}$  |

Notice how we used some shortcuts, like writing "anything" on a single transition rather than creating a transition for every possible input from $s_2$ to $s_2$ or from $s_{err}$ to $s_{err}$. We did something similar for that transition from $s_0$ or $s_1$ to $s_{err}$, labeling a transition in a way that says that any input except for ":" or "-", respectively, transitioned to $s_{err}$. 

***


We can create a class that implements this FSA. We'll define two types of things in the constructor. First, we'll need to define everything required in the definition of a FSA: set of states, set if input characters, start state, set of accept states, and transition function. Second, we'll define some variables that we'll use to manage the input string or that will be used in a subsequent tool.

After the constructor, a method is defined for each state. (Note that this is not the most efficient implementation of a state machine, but it's one of the easiest to understand. If you take CS 340, you'll learn about _design patterns_ that frequently appear in software engineering. One of these design patterns is the _state machine pattern_.) Each method has the same pattern: get the current input and then transitions to the next state determined by the current input. (Ignore the history_message stuff. We'll use it later)



In [14]:
class colonDash_FSA:
    def __init__(self):
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state S0, S1, S2, Serr will be represented by its own method
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state = self.S0  # Initialize the starting state to the correct method
        self.accept_states = set(); self.accept_states.add(self.S2)  
        # Transition function: Each transition will be defined in the state methods
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string = ""      # Default empty input
        self.FSA_name = "colon-dash state machine"
        self.num_chars_read = 0
        self.history = []
            
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def S0(self):
        current_input = self.__getCurrentInput()
        if current_input == ':': 
            next_state = self.S1
            history_message = self.get_History_String("S0",current_input,"S1")
        else: 
            next_state = self.Serr
            history_message = self.get_History_String("S0",current_input,"Serr")
        self.history.append(history_message)
        return next_state
    def S1(self):
        current_input = self.__getCurrentInput()
        if current_input == '-': 
            next_state = self.S2
            history_message = self.get_History_String("S1",current_input,"S2")
        else: 
            next_state = self.Serr
            history_message = self.get_History_String("S1",current_input,"Serr")
        self.history.append(history_message)
        return next_state
    def S2(self):
        current_input = self.__getCurrentInput()
        next_state = self.S2 # loop in state S2
        history_message = self.get_History_String("S2",current_input,"S2")
        self.history.append(history_message)
        return next_state
    def Serr(self):
        current_input = self.__getCurrentInput()
        next_state = self.Serr # loop in state Serr
        history_message = self.get_History_String("Serr",current_input,"Serr")
        self.history.append(history_message)
        return next_state
    
    ############################
    # Manager functions
    ############################
    def Run(self,input_string):
        ###############################################################
        # This function will be called to make the FSA execute
        # It records the input string,
        # sets the current state to the start state
        # and then calls a state method that returns the next state
        # Each state method accesses the input_string
        ###############################################################
            # Remember input_string
        self.input_string = input_string
            # Set current state to start state
        current_state = self.start_state
            # Call current state, which starts the FSA
        while self.num_chars_read < len(self.input_string):
            current_state = current_state()
            # Check whether the FSA ended in an accept state
        outcome = False 
        if current_state in self.accept_states: outcome = True # Accept if the FSA ended in an accept state
        return outcome
    def Reset(self):
        self.num_chars_read = 0
        self.history = []
    
    ############################
    # Public Getters and Setters
    ############################
    def get_Name(self): return self.FSA_name
    def get_History(self): 
        output_string = ""
        for message in self.history:
            output_string = output_string + message + "\n"
        return output_string
    def get_History_String(self,current_state,input,next_state):
        history_message = "From state " + current_state + " read input " + \
            input + " and transitioned to state " + next_state
        return history_message
    
    ############################
    # Private Helper functions
    ############################
    def __getCurrentInput(self):  # The double underscore makes the method private
        current_input = self.input_string[self.num_chars_read]
        self.num_chars_read += 1
        return current_input
    

Let's test whether the FSA we defined actually works by running it on an input

In [20]:
my_FSA = colonDash_FSA()
input_string = ":-"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")


The  colon-dash state machine FSA accepted the input string ' :- '


Yay! it worked. 

It's useful to print out the _trace_ of the FSA. The _trace_ is defined as the history of each state that was visited, the input character read from that state, and the state to which the FSA transitioned given that input.

***


In [23]:
my_FSA.Reset()  # This method resets the FSA to run on a new input.
input_string = ":-"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
print("Here is the trace on input '",input_string,"'")
print(my_FSA.get_History())

The  colon-dash state machine FSA accepted the input string ' :- '
Here is the trace on input ' :- '
From state S0 read input : and transitioned to state S1
From state S1 read input - and transitioned to state S2



Just as expected, the FSA started in the start state, S0, read the colon and transitioned to state S1, and then read the dash and transitioned to state S2. The trace shows the sequence of states visited and inputs read.

***

Let's try an input string that should not be accepted by the FSA.

In [26]:
my_FSA.Reset()
input_string = ":"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
print("Here is the trace on input '",input_string,"'")
print(my_FSA.get_History())

The  colon-dash state machine FSA did not accept the input string ' : '
Here is the trace on input ' : '
From state S0 read input : and transitioned to state S1



***
Let's try an input that is a little longer.

In [25]:
my_FSA.Reset()
input_string = ":::-"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_FSA.get_History())

The  colon-dash state machine FSA did not accept the input string ' :::- '
Trace the following history using the picture of the FSA above:
From state S0 read input : and transitioned to state S1
From state S1 read input : and transitioned to state Serr
From state Serr read input : and transitioned to state Serr
From state Serr read input - and transitioned to state Serr



***
Let's try a long input that should be accepted.

In [28]:
my_FSA.Reset()
input_string = ":-::- testing"
accept_status = my_FSA.Run(input_string)
if accept_status: print("The ", my_FSA.get_Name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_FSA.get_Name(), "FSA did not accept the input string '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_FSA.get_History())

The  colon-dash state machine FSA accepted the input string ' :-::- testing '
Trace the following history using the picture of the FSA above:
From state S0 read input : and transitioned to state S1
From state S1 read input - and transitioned to state S2
From state S2 read input : and transitioned to state S2
From state S2 read input : and transitioned to state S2
From state S2 read input - and transitioned to state S2
From state S2 read input   and transitioned to state S2
From state S2 read input t and transitioned to state S2
From state S2 read input e and transitioned to state S2
From state S2 read input s and transitioned to state S2
From state S2 read input t and transitioned to state S2
From state S2 read input i and transitioned to state S2
From state S2 read input n and transitioned to state S2
From state S2 read input g and transitioned to state S2

